<a href="https://colab.research.google.com/github/gmauricio-toledo/NLP-LCC/blob/main/Tareas%20y%20Actividades/AC%2007%20-%20Likes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad en clase: Regresión y NLP

## Instrucciones:

La pregunta que queremos responder es **¿qué palabras son las más importantes para cuantificar la popularidad de una publicación en redes sociales?**

Para responder esto, realiza los siguientes pasos:

1. Escoge una estrategia de preprocesamiento, limpieza y normalización básica. Aplicala al corpus.
2. Haz la división en entrenamiento y prueba (75%/25%)
3. Realiza la vectorización de los documentos con BOW o TF-IDF
4. Entrenar un modelo de regresión (no necesariamente regresión lineal) en la parte de entrenamiento. Escoge alguno de los modelos de la lista de abajo.
5. Realizar la inferencia sobre el **conjunto de prueba y entrenamiento**
6. Evaluar ambas predicciones usando [MAPE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_percentage_error.html). **Imprimir ambas predicciones**
7. Imprime las 10 features más importantes, según tu modelo.


## Modelos de regresión que proporcionan la importancia de las variables

### Modelos lineales:
* [Regresión Lineal](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html), la importancia de las features es el atributo `coef_`, entre más alto en valor absoluto, mayor la importancia
* [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html), la importancia de las features es el atributo `coef_`, entre más alto en valor absoluto, mayor la importancia
* [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html), la importancia de las features es el atributo `coef_`, los coeficientes en cero indican features eliminadas por el modelo
* [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html), la importancia de las features es el atributo `coef_`, combina penalización L1 y L2, coeficientes en cero son features ignoradas
* [BayesianRidge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.BayesianRidge.html), la importancia de las features es el atributo `coef_`, entre más alto en valor absoluto, mayor la importancia
### Modelos de árbol:
* [DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html), la importancia de las features es el atributo `feature_importances_`, representa la reducción total de impureza aportada por cada feature, entre más alto mayor la importancia
* [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html), la importancia de las features es el atributo `feature_importances_`, promedio de la reducción de impureza a través de todos los árboles, entre más alto mayor la importancia
* [GradientBoostingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html), la importancia de las features es el atributo `feature_importances_`, basada en cuánto contribuye cada feature a reducir el error en los árboles de boosting, entre más alto mayor la importancia
* [HistGradientBoostingRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingRegressor.html), la importancia de las features es el atributo `feature_importances_`, equivalente al GradientBoosting pero más eficiente con datos grandes, entre más alto mayor la importancia
* [ExtraTreesRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesRegressor.html), la importancia de las features es el atributo `feature_importances_`, similar a RandomForest pero con splits aleatorios, entre más alto mayor la importancia
* [AdaBoostRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html), la importancia de las features es el atributo `feature_importances_`, basada en el peso acumulado de cada feature en los estimadores débiles, entre más alto mayor la importancia

🟢 Bajar el corpus

In [1]:
!gdown 12CkPYr94f-K8TyDFf5OiNtWS33eiR96U

Downloading...
From: https://drive.google.com/uc?id=12CkPYr94f-K8TyDFf5OiNtWS33eiR96U
To: /content/post_likes.csv
100% 72.1k/72.1k [00:00<00:00, 13.9MB/s]


🟢 Leer el corpus

In [2]:
import pandas as pd

likes_df = pd.read_csv('post_likes.csv',index_col=0)
likes_df

,Text,Likes
0,Enjoying a beautiful day at the park! ...,30.0
1,Traffic was terrible this morning. ...,10.0
2,Just finished an amazing workout! 💪 ...,40.0
3,Excited about the upcoming weekend getaway! ...,15.0
4,Trying out a new recipe for dinner tonight. ...,25.0
...,...,...
728,Collaborating on a science project that receiv...,39.0
729,Attending a surprise birthday party organized ...,48.0
730,Successfully fundraising for a school charity ...,42.0
731,"Participating in a multicultural festival, cel...",43.0


🟢 Separar los documentos y las etiquetas

In [3]:
docs = likes_df['Text'].values
y = likes_df['Likes'].values

🟢 Exploración del corpus

In [6]:
for doc in docs[:7]:
  print(doc)

 Enjoying a beautiful day at the park!              
 Traffic was terrible this morning.                 
 Just finished an amazing workout! 💪               
 Excited about the upcoming weekend getaway!        
 Trying out a new recipe for dinner tonight.        
 Feeling grateful for the little things in life.    
 Rainy days call for cozy blankets and hot cocoa.   


🔴 Desarrollo de la actividad

---
**Ejemplo** para determinar las palabras con mayor importancias:

In [4]:
# Para modelos de árbol (DecisionTree, GradientBoosting, etc.):
importancias = model.feature_importances_  # importancia de cada feature (palabra)
# Para modelos lineales (LinearRegression, Lasso, etc.):
importancias = np.abs(model.coef_) # importancia de cada feature (palabra)

top10_indices = importancias.argsort()[-10:][::-1]  # índices de las 10 más importantes, de mayor a menor
top10_words = vectorizador.get_feature_names_out()[top10_indices]  # palabras correspondientes
top10_valores = importancias[top10_indices]  # sus valores de importancia

for word, val in zip(top10_words, top10_valores):
    print(f"{word}: {val:.4f}")  # imprime palabra y su importancia